In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-02 05:36:04.069901: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 05:36:04.069966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 05:36:04.073502: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 05:36:04.094683: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 05:36:06.698080: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../HAM10000/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/10015
[INFO]: Processed 2000/10015
[INFO]: Processed 3000/10015
[INFO]: Processed 4000/10015
[INFO]: Processed 5000/10015
[INFO]: Processed 6000/10015
[INFO]: Processed 7000/10015
[INFO]: Processed 8000/10015
[INFO]: Processed 9000/10015
[INFO]: Processed 10000/10015
(10015, 224, 224, 3)
(10015,)


In [8]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
# Extract features
features = model.predict(data, batch_size=32, verbose=1)
 
# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-02 05:41:02.914419: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 36183 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0
2024-04-02 05:41:13.967498: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-02 05:41:14.284176: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


313/313 [==============================] - 9s 19ms/step
(10015, 25088)


In [9]:
data = features_flatten

In [10]:
num_classes = len(np.unique(labels))

In [11]:
import gc
import os

def calculate_sensitivity_specificity(y_true, y_pred, class_label):
    # Create a binary confusion matrix for the specified class
    true_positive = sum((y_true == class_label) & (y_pred == class_label))
    false_positive = sum((y_true != class_label) & (y_pred == class_label))
    true_negative = sum((y_true != class_label) & (y_pred != class_label))
    false_negative = sum((y_true == class_label) & (y_pred != class_label))

    # Calculate sensitivity and specificity
    sensitivity = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0

    return sensitivity, specificity
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":
        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = to_categorical(labels[train_index]), to_categorical(labels[test_index])

                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                #train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                #train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])


                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                # print(y_test)
                # print(y_test_encoded)
                # print(y_pred)
                # print(predictions)
                # Calculate evaluation metrics for the current fold
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivities = []
                specificities = []
                for class_label in range(num_classes):
                    sen, spe = calculate_sensitivity_specificity(y_test_encoded, y_pred, class_label)
                    sensitivities.append(sen)
                    specificities.append(spe)
                sensitivity = sum(sensitivities) / num_classes
                specificity = sum(specificities) / num_classes
                auc_roc = roc_auc_score(y_test, predictions,multi_class='ovr')
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, average='macro')
                f1 = f1_score(y_test_encoded, y_pred, average='macro')
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_prob = model.predict_proba(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivities = []
            specificities = []
            for class_label in range(num_classes):
                sen, spe = calculate_sensitivity_specificity(y_test, y_pred, class_label)
                sensitivities.append(sen)
                specificities.append(spe)
            sensitivity = sum(sensitivities) / num_classes
            specificity = sum(specificities) / num_classes
            auc_roc = roc_auc_score(y_test, y_prob,multi_class='ovr')
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [12]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.779441117764471
Sensitivity: 0.5840285187833161
Specificity: 0.9452773254596785
AUC-ROC: 0.9104764520427041
MCC: 0.574019753289111
Precision: 0.6653447808489358
F1 Score: 0.606279614041189

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.7674650698602794
Sensitivity: 0.5475663812550806
Specificity: 0.9376732152388879
AUC-ROC: 0.925076862040797
MCC: 0.5422651242679825
Precision: 0.6125129407364366
F1 Score: 0.5748033810111519

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.7652347652347652
Sensitivity: 0.5710100978603873
Specificity: 0.9371180947329982
AUC-ROC: 0.927132701241706
MCC: 0.5400824320703564
Precision: 0.6340056202574293
F1 Score: 0.598778202589166

[INFO] Fold 9 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.7592407592407593
Sensitivity: 0.5181130559324869
Specificity: 0.9396052802492576
AUC-ROC: 0.8991529633427867
MCC: 0.532559237362832
Precision: 0.5426911183942086
F1 Score: 0.5200454370425558

[INFO] Fold 10 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.7862137862137862
Sensitivity: 0.5433763910842802
Specificity: 0.9392100152368046
AUC-ROC: 0.9174284783275082
MCC: 0.5705850434683879
Precision: 0.6828976191792588
F1 Score: 0.5892721857858044


In [13]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.3992015968063872
Sensitivity: 0.23194452448183792
Specificity: 0.8775189225001764
AUC-ROC: 0.5558452204587485
MCC: 0.11069371237407948
Precision: 0.23285439780044095
F1 Score: 0.21133726034407355

[INFO] Fold 2 / 10 for NB


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.38922155688622756
Sensitivity: 0.19734427868756227
Specificity: 0.8703770425999714
AUC-ROC: 0.5364319814524817
MCC: 0.07284764439795975
Precision: 0.18310860244804994
F1 Score: 0.17925678394011238

[INFO] Fold 3 / 10 for NB
Accuracy: 0.4041916167664671
Sensitivity: 0.2034703900375542
Specificity: 0.8747335041443104
AUC-ROC: 0.5399982681794538
MCC: 0.09702868416446758
Precision: 0.18737100824809735
F1 Score: 0.1840436273310007

[INFO] Fold 4 / 10 for NB
Accuracy: 0.4031936127744511
Sensitivity: 0.2139744839591196
Specificity: 0.8734054991101268
AUC-ROC: 0.5451820467584613
MCC: 0.09456754173918899
Precision: 0.21182993218360877
F1 Score: 0.19903298423322427

[INFO] Fold 5 / 10 for NB
Accuracy: 0.40219560878243515
Sensitivity: 0.20177313846552034
Specificity: 0.8727124467489491
AUC-ROC: 0.5382333269784819
MCC: 0.088747173112312
Precision: 0.18382826717472714
F1 Score: 0.18067474854557528

[INFO] Fold 6 / 10 for NB
Accuracy: 0.4065934065934066
Sensitivity: 0.20044500969422074
S

/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.4155844155844156
Sensitivity: 0.18869806229979805
Specificity: 0.8734938857149152
AUC-ROC: 0.5315334256536831
MCC: 0.08241889171340497
Precision: 0.17704817498024666
F1 Score: 0.1737902723985178

[INFO] Fold 10 / 10 for NB
Accuracy: 0.3706293706293706
Sensitivity: 0.22081912137882287
Specificity: 0.8709930448997361
AUC-ROC: 0.5485955489769074
MCC: 0.07930481002013508
Precision: 0.2002458936780403
F1 Score: 0.19481905642909877


In [14]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.687624750499002
Sensitivity: 0.27932013337130607
Specificity: 0.9163727709876658
AUC-ROC: 0.7519487237577247
MCC: 0.37014457023535424
Precision: 0.46384393619065917
F1 Score: 0.2882921078838042

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.7145708582834331
Sensitivity: 0.32197200885900246
Specificity: 0.920630983982684
AUC-ROC: 0.7672455926864928
MCC: 0.41956705463364696
Precision: 0.4992402931801597
F1 Score: 0.3398749641445806

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.7105788423153693
Sensitivity: 0.3040099530504648
Specificity: 0.9188824281298107
AUC-ROC: 0.7433050732659762
MCC: 0.4111343013611628
Precision: 0.5313275093100254
F1 Score: 0.3141066275111579

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.6826347305389222
Sensitivity: 0.29120951637099884
Specificity: 0.9111905307252035
AUC-ROC: 0.7578553006641331
MCC: 0.3578325844443345
Precision: 0.5088636593537865
F1 Score: 0.3044218227745716

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.7125748502994012
Sensitivity

/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7072927072927073
Sensitivity: 0.3166191797942039
Specificity: 0.9175376758751257
AUC-ROC: 0.7412344741984163
MCC: 0.39717495415851134
Precision: 0.502221893788862
F1 Score: 0.3394435559380202

[INFO] Fold 10 / 10 for KNN
Accuracy: 0.7002997002997003
Sensitivity: 0.29265806251413923
Specificity: 0.9164002430802817
AUC-ROC: 0.7592697030088126
MCC: 0.3816036156539785
Precision: 0.46078589505084466
F1 Score: 0.3036320251858296


In [15]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7105788423153693
Sensitivity: 0.20657859016067973
Specificity: 0.8876484353331273
AUC-ROC: 0.849603694830651
MCC: 0.31231512957125784
Precision: 0.3466038611413061
F1 Score: 0.21450846300694987

[INFO] Fold 2 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6996007984031936
Sensitivity: 0.20144845645911744
Specificity: 0.8810345618308553
AUC-ROC: 0.8911721583850868
MCC: 0.2687131873784028
Precision: 0.5231499036167866
F1 Score: 0.21256517191352373

[INFO] Fold 3 / 10 for Random Forest


...

Accuracy: 0.6953046953046953
Sensitivity: 0.18993588009981452
Specificity: 0.879138810974017
AUC-ROC: 0.8701167196175479
MCC: 0.24545950091012633
Precision: 0.3742332842717281
F1 Score: 0.1943452380952381

[INFO] Fold 9 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6983016983016983
Sensitivity: 0.20994560792053557
Specificity: 0.880709899061959
AUC-ROC: 0.8630872422423509
MCC: 0.2613274610695969
Precision: 0.5299767728447294
F1 Score: 0.22864996022598688

[INFO] Fold 10 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6903096903096904
Sensitivity: 0.19325103909325655
Specificity: 0.8767963299380475
AUC-ROC: 0.8748411240347582
MCC: 0.2263127962368524
Precision: 0.36116056804210117
F1 Score: 0.20215572476753643


In [16]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 4.345489 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2921771
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 21934
[LightGBM] [Info] Start training from score -3.422843
[LightGBM] [Info] Start training from score -2.970858
[LightGBM] [Info] Start training from score -2.209729
[LightGBM] [Info] Start training from score -4.471694
[LightGBM] [Info] Start training from score -2.196670
[LightGBM] [Info] Start training from score -0.401092
[LightGBM] [Info] Start training from score -4.254393
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.781437125748503
Sensitivity: 0.4091454608512177
Specificity: 0.9304695230012883
AUC-ROC: 0.9211462219976639
MCC: 0.5395699519008768
Precision: 0.616682520871727
F1 Score: 0.4661016556814142

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 4.502028 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2916129
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 21923
[LightGBM] [Info] Start training from score -3.422843
[LightGBM] [Info] Start training from score -2.970858
[LightGBM] [Info] Start training from score -2.209729
[LightGBM] [Info] Start training from score -4.471694
[LightGBM] [Info] Start training from score -2.196670
[LightGBM] [Info] Start training from score -0.401092
[LightGBM] [Info] Start training from score -4.254393
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7654690618762475
Sensitivity: 0.38915807966980676
Specificity: 0.9189902513809646
AUC-ROC: 0.9389759297234536
MCC: 0.49365938119423997
Precision: 0.6209171291477682
F1 Score: 0.4495649980438491

[INFO] Fold 3 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 4.402319 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2919981
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 21943
[LightGBM] [Info] Start training from score -3.422843
[LightGBM] [Info] Start training from score -2.970858
[LightGBM] [Info] Start training from score -2.209729
[LightGBM] [Info] Start training from score -4.471694
[LightGBM] [Info] Start training from score -2.196670
[LightGBM] [Info] Start training from score -0.401092
[LightGBM] [Info] Start training from score -4.254393
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


...

Accuracy: 0.7672327672327672
Sensitivity: 0.41495294849320885
Specificity: 0.9207410631144122
AUC-ROC: 0.9327167271406507
MCC: 0.4999449574843342
Precision: 0.6686069576778344
F1 Score: 0.4754821649098568

[INFO] Fold 9 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 4.471523 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2918541
[LightGBM] [Info] Number of data points in the train set: 9014, number of used features: 21933
[LightGBM] [Info] Start training from score -3.419559
[LightGBM] [Info] Start training from score -2.968807
[LightGBM] [Info] Start training from score -2.209840
[LightGBM] [Info] Start training from score -4.462143
[LightGBM] [Info] Start training from score -2.196781
[LightGBM] [Info] Start training from score -0.401369
[LightGBM] [Info] Start training from score -4.262347
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7642357642357642
Sensitivity: 0.38615982472660393
Specificity: 0.9188903990812597
AUC-ROC: 0.9300992427536673
MCC: 0.48852677862765115
Precision: 0.6496488361611341
F1 Score: 0.45346666243134465

[INFO] Fold 10 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 4.640282 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2922229
[LightGBM] [Info] Number of data points in the train set: 9014, number of used features: 21958
[LightGBM] [Info] Start training from score -3.419559
[LightGBM] [Info] Start training from score -2.970969
[LightGBM] [Info] Start training from score -2.209840
[LightGBM] [Info] Start training from score -4.471805
[LightGBM] [Info] Start training from score -2.196781
[LightGBM] [Info] Start training from score -0.401203
[LightGBM] [Info] Start training from score -4.254504
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7622377622377622
Sensitivity: 0.35556324594171074
Specificity: 0.9174176883810937
AUC-ROC: 0.9269678024104849
MCC: 0.4812009399086202
Precision: 0.6534871518544204
F1 Score: 0.418546780411563


In [17]:
# Save results to a CSV file
results_df.to_csv('DS7_VGG16.csv', index=False)

In [18]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.773242,0.540096,0.939233,0.915052,0.551633,0.628060,0.568749,-547.125000,2075.885579
1,NB,0.396304,0.209147,0.873710,0.542614,0.091575,0.196720,0.188579,863.351562,83.255250
2,KNN,0.701148,0.307597,0.916057,0.759078,0.388282,0.518172,0.325930,873.464844,137.501791
3,Random Forest,0.701046,0.204104,0.881281,0.865124,0.271050,0.448762,0.216601,899.691406,141.184834
4,LGBM,0.769046,0.389262,0.921430,0.928921,0.502702,0.626768,0.448608,1262.609375,12130.668015
